<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/010_Mask2Former_ObtenerDatos_Avanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mahotas scikit-image opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import cv2
from PIL import Image
import mahotas as mh
from skimage import feature, filters, segmentation, measure
from skimage.color import rgb2gray, rgb2hsv, rgb2lab
from sklearn.cluster import KMeans
from typing import Dict, Any
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class ExtractorCaracteristicasAvanzado:
    """
    Extractor de características de imágenes usando librerías especiales
    para análisis completo.
    """

    def __init__(self):
        print("🔧 Inicializando Extractor de Características Avanzado")
        print("   📚 Librerías: Mahotas, Scikit-image, OpenCV")

    def analizar_imagen_completa(self, imagen: Image.Image, ruta: str) -> Dict[str, Any]:
        """
        Análisis completo usando librerías especializadas

        Args:
            imagen: Imagen PIL en formato RGB
            ruta: Ruta del archivo original

        Returns:
            Diccionario completo con características extraídas
        """
        # Conversiones básicas
        img_array = np.array(imagen)
        img_gray = rgb2gray(img_array)
        img_hsv = rgb2hsv(img_array)
        img_lab = rgb2lab(img_array)

        # Convertir a uint8 para mahotas
        img_gray_uint8 = (img_gray * 255).astype(np.uint8)

        caracteristicas = {
            'metadatos_basicos': self._extraer_metadatos_basicos(imagen, ruta),
            'color_y_paleta': self._analizar_color_avanzado(img_array, img_hsv, img_lab),
            'texturas_mahotas': self._extraer_texturas_mahotas(img_gray_uint8),
            'texturas_skimage': self._extraer_texturas_skimage(img_gray),
            'caracteristicas_geometricas': self._analizar_geometria_avanzada(img_gray_uint8),
            'multiscale_features': self._extraer_multiscale_features(img_gray),
            'propiedades_regionales': self._analizar_propiedades_regionales(img_gray),
            'descriptores_locales': self._extraer_descriptores_locales(img_gray_uint8)
        }

        return caracteristicas

    def _extraer_metadatos_basicos(self, imagen: Image.Image, ruta: str) -> Dict:
        """Metadatos básicos de la imagen"""
        w, h = imagen.size
        return {
            'dimensiones': {'ancho': w, 'alto': h},
            'aspecto_ratio': round(w / h, 3),
            'megapixeles': round((w * h) / 1000000, 2),
            'orientacion': 'horizontal' if w > h else 'vertical' if h > w else 'cuadrada',
            'formato': ruta.split('.')[-1].lower() if '.' in ruta else 'desconocido'
        }

    def _analizar_color_avanzado(self, img_rgb: np.ndarray, img_hsv: np.ndarray, img_lab: np.ndarray) -> Dict:
        """Análisis de color usando sklearn para paleta dominante"""
        # Paleta dominante con K-means
        pixels = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
        kmeans.fit(pixels)

        colores_dominantes = kmeans.cluster_centers_.astype(int).tolist()
        proporciones = np.bincount(kmeans.labels_) / len(kmeans.labels_)

        # Estadísticas básicas por canal
        stats_color = {
            'paleta_dominante': colores_dominantes,
            'proporciones_colores': proporciones.tolist(),
            'estadisticas_rgb': {
                'media': np.mean(img_rgb, axis=(0,1)).tolist(),
                'std': np.std(img_rgb, axis=(0,1)).tolist(),
                'rango': {
                    'min': np.min(img_rgb, axis=(0,1)).tolist(),
                    'max': np.max(img_rgb, axis=(0,1)).tolist()
                }
            },
            'hsv_global': {
                'hue_medio': float(np.mean(img_hsv[:,:,0])),
                'saturacion_media': float(np.mean(img_hsv[:,:,1])),
                'valor_medio': float(np.mean(img_hsv[:,:,2]))
            },
            'lab_luminancia': {
                'L_medio': float(np.mean(img_lab[:,:,0])),
                'a_medio': float(np.mean(img_lab[:,:,1])),
                'b_medio': float(np.mean(img_lab[:,:,2]))
            }
        }

        return stats_color

    def _extraer_texturas_mahotas(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Mahotas"""
        try:
            # Características Haralick (muy utilizadas en análisis de texturas)
            haralick_features = mh.features.haralick(img_gray, return_mean=True)

            # Local Binary Pattern
            lbp = mh.features.lbp(img_gray, radius=1, points=8, ignore_zeros=False)

            # Características Zernike (descriptores de forma)
            try:
                zernike_features = mh.features.zernike_moments(img_gray, radius=21)
            except:
                zernike_features = np.zeros(25)  # Fallback si falla

            # Threshold de Otsu
            otsu_threshold = mh.otsu(img_gray)

            # Características adicionales
            pftas = mh.features.pftas(img_gray)  # Parameter-free threshold adjacency statistics

            return {
                'haralick_features': haralick_features.tolist(),
                'lbp_histogram': np.histogram(lbp, bins=50)[0].tolist(),
                'zernike_moments': zernike_features.tolist(),
                'otsu_threshold': float(otsu_threshold),
                'pftas': pftas.tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en Mahotas: {str(e)}",
                'haralick_features': [],
                'lbp_histogram': [],
                'zernike_moments': [],
                'otsu_threshold': 0.0,
                'pftas': []
            }

    def _extraer_texturas_skimage(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Scikit-image"""
        try:
            # Local Binary Pattern con scikit-image
            lbp_skimage = feature.local_binary_pattern(img_gray, P=8, R=1, method='uniform')

            # GLCM (Gray Level Co-occurrence Matrix) features
            img_scaled = (img_gray * 255).astype(np.uint8)
            glcm = feature.graycomatrix(img_scaled, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                      levels=256, symmetric=True, normed=True)

            # Propiedades GLCM
            contrast = feature.graycoprops(glcm, 'contrast').mean()
            dissimilarity = feature.graycoprops(glcm, 'dissimilarity').mean()
            homogeneity = feature.graycoprops(glcm, 'homogeneity').mean()
            energy = feature.graycoprops(glcm, 'energy').mean()
            correlation = feature.graycoprops(glcm, 'correlation').mean()

            return {
                'lbp_uniform_histogram': np.histogram(lbp_skimage, bins=10)[0].tolist(),
                'glcm_properties': {
                    'contrast': float(contrast),
                    'dissimilarity': float(dissimilarity),
                    'homogeneity': float(homogeneity),
                    'energy': float(energy),
                    'correlation': float(correlation)
                }
            }

        except Exception as e:
            return {
                'error': f"Error en Scikit-image: {str(e)}",
                'lbp_uniform_histogram': [],
                'glcm_properties': {}
            }

    def _analizar_geometria_avanzada(self, img_gray: np.ndarray) -> Dict:
        """Análisis geométrico usando mahotas y scikit-image"""
        try:
            # Detección de bordes con diferentes métodos
            edges_canny = feature.canny(img_gray / 255.0)

            # Usando mahotas para bordes
            edges_sobel = mh.sobel(img_gray)

            # Detección de esquinas
            corners = feature.corner_harris(img_gray / 255.0)
            corner_peaks = feature.corner_peaks(corners, min_distance=5)

            # Análisis de forma usando momentos
            moments = measure.moments(img_gray)
            centroid = measure.centroid(img_gray)

            return {
                'bordes_canny': float(np.sum(edges_canny)),
                'bordes_sobel_intensidad': float(np.mean(edges_sobel)),
                'num_corners': len(corner_peaks),
                'centroide': [float(centroid[0]), float(centroid[1])],
                'momentos_hu': measure.moments_hu(moments).tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en análisis geométrico: {str(e)}",
                'bordes_canny': 0.0,
                'bordes_sobel_intensidad': 0.0,
                'num_corners': 0,
                'centroide': [0.0, 0.0],
                'momentos_hu': []
            }

    def _extraer_multiscale_features(self, img_gray: np.ndarray) -> Dict:
        """Características multi-escala usando scikit-image"""
        try:
            # Características básicas multi-escala
            features_multiscale = feature.multiscale_basic_features(
                img_gray,
                intensity=True,
                edges=True,
                texture=True,
                sigma_min=0.5,
                sigma_max=8
            )

            # Estadísticas de las características multi-escala
            return {
                'num_features': features_multiscale.shape[-1],
                'feature_means': np.mean(features_multiscale, axis=(0,1)).tolist(),
                'feature_stds': np.std(features_multiscale, axis=(0,1)).tolist()
            }

        except Exception as e:
            return {
                'error': f"Error en features multiscale: {str(e)}",
                'num_features': 0,
                'feature_means': [],
                'feature_stds': []
            }

    def _analizar_propiedades_regionales(self, img_gray: np.ndarray) -> Dict:
        """Análisis de propiedades regionales usando segmentación"""
        try:
            # Segmentación usando SLIC
            segments = segmentation.slic(img_gray, n_segments=100, compactness=10)

            # Propiedades de regiones
            regions = measure.regionprops(segments, intensity_image=img_gray)

            if regions:
                areas = [r.area for r in regions]
                eccentricities = [r.eccentricity for r in regions]
                intensities = [r.mean_intensity for r in regions]

                return {
                    'num_regiones': len(regions),
                    'area_promedio': float(np.mean(areas)),
                    'excentricidad_promedio': float(np.mean(eccentricities)),
                    'intensidad_promedio_regiones': float(np.mean(intensities)),
                    'variabilidad_areas': float(np.std(areas)),
                    'variabilidad_intensidades': float(np.std(intensities))
                }
            else:
                return {'num_regiones': 0}

        except Exception as e:
            return {
                'error': f"Error en análisis regional: {str(e)}",
                'num_regiones': 0
            }

    def _extraer_descriptores_locales(self, img_gray: np.ndarray) -> Dict:
        """Descriptores locales usando OpenCV integrado"""
        try:
            # Conversión para OpenCV
            img_cv = img_gray.copy()

            # Detección de puntos clave con diferentes detectores
            # ORB (orientación y escala invariante)
            orb = cv2.ORB_create(nfeatures=100)
            keypoints_orb = orb.detect(img_cv, None)

            # FAST corners
            fast = cv2.FastFeatureDetector_create()
            keypoints_fast = fast.detect(img_cv, None)

            # BRIEF descriptors (si hay keypoints)
            brief = cv2.xfeatures2d.BriefDescriptorExtractor_create() if hasattr(cv2, 'xfeatures2d') else None

            return {
                'orb_keypoints': len(keypoints_orb),
                'fast_keypoints': len(keypoints_fast),
                'keypoint_density': (len(keypoints_orb) + len(keypoints_fast)) / (img_gray.shape[0] * img_gray.shape[1])
            }

        except Exception as e:
            return {
                'error': f"Error en descriptores locales: {str(e)}",
                'orb_keypoints': 0,
                'fast_keypoints': 0,
                'keypoint_density': 0.0
            }

# Clase integrada mejorada que usa el extractor avanzado
class ProcesadorResultadosConLibrerias(ProcesadorResultados):
    """
    Versión optimizada usando librerías especializadas
    """

    def __init__(self, output_path: str):
        super().__init__(output_path)
        self.extractor = ExtractorCaracteristicasAvanzado()
        print("✅ Procesador con librerías especializadas inicializado")

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonas,
                       umbrales: List[float]) -> Optional[Dict]:
        """
        Procesamiento con características extraídas usando librerías especializadas
        """
        inicio = time.time()

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_img = Utils.calcular_hash(ruta_imagen)

            # Extracción completa de características usando librerías
            print(f"   🔍 Extrayendo características avanzadas...")
            caracteristicas_avanzadas = self.extractor.analizar_imagen_completa(imagen, ruta_imagen)

            # Detección de personas (original)
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            tiempo_total = (time.time() - inicio) * 1000

            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {
                    'archivo': os.path.basename(ruta_imagen),
                    'hash': hash_img,
                    'ruta_completa': ruta_imagen,
                    'caracteristicas_avanzadas': caracteristicas_avanzadas  # 🔄 NUEVO
                },
                'deteccion': resultados_deteccion,
                'tiempo_total_ms': tiempo_total,
                'modelo': detector.modelo_info,
                'version_extractor': 'librerias_especializadas',  # 🔄 NUEVO
                'exitoso': True
            }

        except Exception as e:
            return {
                'timestamp': datetime.now().isoformat(),
                'imagen': {
                    'archivo': os.path.basename(ruta_imagen),
                    'ruta_completa': ruta_imagen
                },
                'error': str(e),
                'exitoso': False
            }

    def mostrar_resumen_avanzado(self, resultados: List[Dict], umbral_config: str) -> None:
        """
        Resumen que incluye estadísticas de las características extraídas
        """
        # Resumen original
        super().mostrar_resumen(resultados, umbral_config)

        exitosos = [r for r in resultados if r.get('exitoso', False)]
        if not exitosos:
            return

        print(f"\n🧬 ANÁLISIS DE CARACTERÍSTICAS AVANZADAS:")

        # Recopilar estadísticas de características
        haralick_means = []
        glcm_contrasts = []
        num_corners = []

        for r in exitosos:
            carac = r['imagen'].get('caracteristicas_avanzadas', {})

            # Características Haralick
            texturas_mh = carac.get('texturas_mahotas', {})
            if 'haralick_features' in texturas_mh and texturas_mh['haralick_features']:
                haralick_means.append(np.mean(texturas_mh['haralick_features']))

            # GLCM contrast
            texturas_sk = carac.get('texturas_skimage', {})
            glcm_props = texturas_sk.get('glcm_properties', {})
            if 'contrast' in glcm_props:
                glcm_contrasts.append(glcm_props['contrast'])

            # Corners
            geom = carac.get('caracteristicas_geometricas', {})
            if 'num_corners' in geom:
                num_corners.append(geom['num_corners'])

        if haralick_means:
            print(f"   📊 Haralick promedio: {np.mean(haralick_means):.3f} ± {np.std(haralick_means):.3f}")
        if glcm_contrasts:
            print(f"   ⚡ GLCM Contrast: {np.mean(glcm_contrasts):.3f} ± {np.std(glcm_contrasts):.3f}")
        if num_corners:
            print(f"   📐 Esquinas promedio: {np.mean(num_corners):.1f} ± {np.std(num_corners):.1f}")

        print(f"   🔬 Usando: Mahotas + Scikit-image + OpenCV")

# Función principal actualizada que mantiene compatibilidad con el código original
def ejecutar_con_librerias_especializadas(modelo_idx: int = 0, umbral_config: str = 'alta_sensibilidad'):
    """
    Función principal que usa librerías especializadas para extracción de características
    """
    # Validaciones (iguales que antes)
    if modelo_idx >= len(config.MODELOS_INFO):
        print(f"❌ Modelo index inválido. Máximo: {len(config.MODELOS_INFO)-1}")
        return

    if umbral_config not in config.UMBRALES:
        print(f"❌ Configuración inválida. Disponibles: {list(config.UMBRALES.keys())}")
        return

    modelo_info = config.MODELOS_INFO[modelo_idx]
    umbral_info = config.UMBRALES[umbral_config]

    print(f"\n{'='*100}")
    print(f"🚀 EVALUACIÓN CON LIBRERÍAS ESPECIALIZADAS")
    print(f"{'='*100}")
    print(f"📋 CONFIGURACIÓN:")
    print(f"   🤖 Modelo: {modelo_info['nombre_corto']}")
    print(f"   🔧 Tipo: {modelo_info['tipo'].upper()}")
    print(f"   📚 Librerías: Mahotas + Scikit-image + OpenCV")
    print(f"   ⚙️  Umbrales: {umbral_info['descripcion']}")

    # Cargar imágenes
    imagenes = Utils.cargar_imagenes(config.DATASET_PATH)
    if not imagenes:
        print("❌ No se encontraron imágenes para procesar")
        return

    umbrales = umbral_info['valores']

    # Usar procesador con librerías especializadas
    detector = DetectorPersonas(modelo_info)
    procesador = ProcesadorResultadosConLibrerias(config.OUTPUT_PATH)  # 🔄 CAMBIO CLAVE

    print(f"{'='*100}")
    print(f"🔄 PROCESANDO {len(imagenes)} IMÁGENES")
    print(f"{'='*100}")

    resultados = []
    tiempo_inicio_total = time.time()

    for i, ruta in enumerate(tqdm(imagenes, desc="Procesando con librerías especializadas")):
        nombre_archivo = os.path.basename(ruta)
        print(f"\n📷 [{i+1:3d}/{len(imagenes):3d}] {nombre_archivo}")

        resultado = procesador.procesar_imagen(ruta, detector, umbrales)

        if resultado and resultado.get('exitoso', False):
            resultados.append(resultado)

            # Mostrar info básica
            deteccion = resultado['deteccion']
            mejor_umbral = min(umbrales)
            datos = deteccion.get(f'umbral_{mejor_umbral}', {})
            personas = datos.get('personas', 0)
            tiempo_img = deteccion.get('tiempo_inferencia_ms', 0)

            print(f"   ✅ {personas:2d} personas | {tiempo_img:.1f}ms | Características ✓")

    # Guardar resultados
    tiempo_total = time.time() - tiempo_inicio_total
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"especializado_{Utils.crear_nombre_archivo(modelo_info, umbral_config, timestamp)}"
    archivo_completo = config.OUTPUT_PATH / nombre_archivo

    metadatos = {
        'metadatos': {
            'version': 'librerias_especializadas_mahotas_skimage',
            'modelo': modelo_info,
            'configuracion_umbrales': {
                'nombre': umbral_config,
                'descripcion': umbral_info['descripcion'],
                'valores': umbral_info['valores']
            },
            'timestamp': timestamp,
            'tiempo_total_segundos': tiempo_total,
            'total_imagenes': len(imagenes),
            'imagenes_exitosas': len(resultados),
            'librerias_usadas': ['mahotas', 'scikit-image', 'opencv-python']
        },
        'resultados': resultados
    }

    Utils.guardar_json(metadatos, str(archivo_completo))

    # Mostrar resumen avanzado
    procesador.mostrar_resumen_avanzado(resultados, umbral_config)

    print(f"\n⏱️  TIEMPO TOTAL: {tiempo_total:.1f} segundos")
    print(f"📁 Archivo guardado como: {nombre_archivo}")
    print(f"🎯 Características extraídas: Haralick, LBP, GLCM, Zernike, Multi-scale")

    detector.limpiar()
    return str(archivo_completo)

### FUNCIÓN MAIN MEJORADA - IGUAL ESTRUCTURA QUE TU ORIGINAL ###
def main_con_caracteristicas_avanzadas():
    """
    Función main que mantiene la misma estructura que tu código original
    pero usa las librerías especializadas para extracción de características
    """
    print(f"\n{'='*100}")
    print(f"🎯 EVALUACIÓN COMPLETA MASK2FORMER + CARACTERÍSTICAS AVANZADAS")
    print(f"{'='*100}")
    print(f"📊 Modelos disponibles: {len(config.MODELOS_INFO)}")
    print(f"⚙️  Configuraciones de umbral: {len(config.UMBRALES)}")
    print(f"🔬 Características: Mahotas + Scikit-image + OpenCV")

    total_combinaciones = len(config.MODELOS_INFO) * len(config.UMBRALES)
    print(f"🔄 Total de combinaciones: {total_combinaciones}")

    combinacion_actual = 0

    try:
        for i, modelo_info in enumerate(config.MODELOS_INFO):
            for umbral_config in config.UMBRALES.keys():
                combinacion_actual += 1

                print(f"\n{'='*100}")
                print(f"🔄 COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                print(f"{'='*100}")

                # 🔄 CAMBIO PRINCIPAL: Usar la nueva función con características
                resultado = ejecutar_con_librerias_especializadas(i, umbral_config)

                if resultado:
                    print(f"✅ Combinación {combinacion_actual} completada exitosamente")
                    print(f"📁 Archivo: {os.path.basename(resultado)}")
                else:
                    print(f"❌ Error en combinación {combinacion_actual}")

                # Pausa entre combinaciones para liberar memoria (igual que antes)
                if combinacion_actual < total_combinaciones:
                    print(f"⏳ Pausa de 3 segundos para liberar memoria...")
                    time.sleep(3)

    except KeyboardInterrupt:
        print(f"\n\n⚠️  EVALUACIÓN INTERRUMPIDA POR EL USUARIO")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")
    except Exception as e:
        print(f"\n❌ ERROR DURANTE LA EVALUACIÓN: {str(e)}")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")

    print(f"\n🎉 EVALUACIONES FINALIZADAS")
    print(f"📁 Todos los resultados guardados en: {config.OUTPUT_PATH}")
    print(f"🔬 Con características avanzadas extraídas usando librerías especializadas")

### COMPATIBILIDAD: Mantener función original disponible ###
def main_original():
    """
    Tu función main original sin modificaciones (por si quieres comparar)
    """
    print(f"\n{'='*100}")
    print(f"🎯 EVALUACIÓN COMPLETA MASK2FORMER (VERSIÓN ORIGINAL)")
    print(f"{'='*100}")
    print(f"📊 Modelos disponibles: {len(config.MODELOS_INFO)}")
    print(f"⚙️  Configuraciones de umbral: {len(config.UMBRALES)}")

    total_combinaciones = len(config.MODELOS_INFO) * len(config.UMBRALES)
    print(f"🔄 Total de combinaciones: {total_combinaciones}")

    combinacion_actual = 0

    try:
        for i, modelo_info in enumerate(config.MODELOS_INFO):
            for umbral_config in config.UMBRALES.keys():
                combinacion_actual += 1

                print(f"\n{'='*100}")
                print(f"🔄 COMBINACIÓN {combinacion_actual}/{total_combinaciones}")
                print(f"{'='*100}")

                # Usar tu función original
                resultado = ejecutar_evaluacion_basica(i, umbral_config)

                if resultado:
                    print(f"✅ Combinación {combinacion_actual} completada exitosamente")
                else:
                    print(f"❌ Error en combinación {combinacion_actual}")

                # Pausa entre combinaciones para liberar memoria
                if combinacion_actual < total_combinaciones:
                    print(f"⏳ Pausa de 3 segundos para liberar memoria...")
                    time.sleep(3)

    except KeyboardInterrupt:
        print(f"\n\n⚠️  EVALUACIÓN INTERRUMPIDA POR EL USUARIO")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")
    except Exception as e:
        print(f"\n❌ ERROR DURANTE LA EVALUACIÓN: {str(e)}")
        print(f"📊 Progreso: {combinacion_actual}/{total_combinaciones} combinaciones completadas")

    print(f"\n🎉 EVALUACIONES FINALIZADAS")
    print(f"📁 Todos los resultados guardados en: {config.OUTPUT_PATH}")

### FUNCIÓN MAIN PRINCIPAL - LA QUE DEBES USAR ###
def main():
    """
    Función main principal que permite elegir entre versiones
    """
    print(f"\n{'='*100}")
    print(f"🚀 SISTEMA DE EVALUACIÓN MASK2FORMER")
    print(f"{'='*100}")

    print("Seleccione el modo de ejecución:")
    print("1. ✨ CON características avanzadas (Recomendado)")
    print("2. 📊 Versión original (solo detección)")
    print("3. 🔬 Ejecutar ambas versiones")

    # Para uso en Colab/Jupyter, puedes cambiar esto por un valor fijo
    # Ejemplo: opcion = "1"  # Para ejecutar siempre con características avanzadas

    try:
        opcion = input("\nIngrese su opción (1, 2, o 3): ").strip()
    except:
        # Si no hay input (como en algunos entornos), usar opción por defecto
        opcion = "1"
        print("Usando opción por defecto: Características avanzadas")

    if opcion == "1":
        print(f"\n🚀 Ejecutando con CARACTERÍSTICAS AVANZADAS...")
        main_con_caracteristicas_avanzadas()

    elif opcion == "2":
        print(f"\n📊 Ejecutando VERSIÓN ORIGINAL...")
        main_original()

    elif opcion == "3":
        print(f"\n🔬 Ejecutando AMBAS VERSIONES...")
        print(f"\n{'='*50} FASE 1: CARACTERÍSTICAS AVANZADAS {'='*50}")
        main_con_caracteristicas_avanzadas()

        print(f"\n{'='*50} FASE 2: VERSIÓN ORIGINAL {'='*50}")
        main_original()

    else:
        print("❌ Opción inválida. Ejecutando versión con características avanzadas por defecto...")
        main_con_caracteristicas_avanzadas()